# 拓扑排序介绍

## 基本概念

> 在图论中，拓扑排序（Topological Sorting）是一个有向无环图（DAG, Directed Acyclic Graph）的所有顶点的线性序列。且该序列必须满足下面两个条件：
>    1. 每个顶点出现且只出现一次。
>    2. 若存在一条从顶点 A 到顶点 B 的路径，那么在序列中顶点 A 出现在顶点 B 的前面。
>
> 注：有向无环图（DAG）才有拓扑排序，非DAG图没有拓扑排序一说。
> 有向无环图的定义为：如果一个有向图从任意顶点出发无法经过若干条边回到这个点，则称这个图是一个有向无环图。

## 实现原理

### Kahn's algorithm

![拓扑排序原理图](http://img.blog.csdn.net/20150507001759702)
+ 该方法实际上属于一种广度优先搜索的方式，可以使用队列实现。
> 1. 从DAG图中选择一个没有前驱（即入度为0）的顶点并输出。
> 2. 从图中删除该顶点和所有以它为起点的有向边。
> 3. 重复1和2直到当前的DAG图为空或当前图中不存在无前驱的顶点为止。

### 深度优先搜索(DFS)：

<img src=https://media.geeksforgeeks.org/wp-content/cdn-uploads/20190704153933/TopologicalSorting1.png width=80%></img>

+ 在使用DFS的方式遍历图时，可以按照一定的顺序将图中的顶点打印出来，为了实现打印出来顶点的拓扑排序，需要在一个顶点的邻居点之前打印它。

## 参考链接

+ https://zhuanlan.zhihu.com/p/56024487
+ https://songlee24.github.io/2015/05/07/topological-sorting/
+ https://www.geeksforgeeks.org/topological-sorting/

In [2]:
# 有向图的定义，使用邻接表保存（有无环需要自己检测）。
class DirectedGraph():
    # n为图中顶点个数。
    def __init__(self, n):
        self.adjtab = [[] for _ in range(n)]
    
    # u为边的起点，v为边的终点。
    def add_edge(self, u, v):
        self.adjtab[u].append(v)

In [3]:
from queue import Queue

# Kahn's algorithm实现
def topologicalSortingKahns(graph):
    # 计算每个顶点的入度。
    n = len(graph.adjtab)
    degree = [0 for _ in range(n)]
    for u in range(n):
        for v in graph.adjtab[u]:
            degree[v] += 1
    que = Queue()
    # 将入度为0的点放入队列。
    for u in range(n):
        if degree[u] == 0:
            que.put(u)
    # 取点，删边，加点。
    sort_res = list()
    while not que.empty():
        u = que.get()
        for v in graph.adjtab[u]:
            degree[v] -= 1
            if degree[v] == 0:
                que.put(v)
        sort_res.append(u)
    return sort_res

In [7]:
# Depth first search拓扑排序实现
def topologicalSortingDfsUnit(graph, u, visited, stack):
    visited[u] = True
    for v in graph.adjtab[u]:
        if not visited[v]:
            topologicalSortingDfsUnit(graph, v, visited, stack)
    stack.append(u)

def topologicalSortingDfs(graph):
    n = len(graph.adjtab)
    visited = [False for _ in range(n)]
    stack = list()
    for u in range(n):
        if not visited[u]:
            topologicalSortingDfsUnit(graph, u, visited, stack)
    return stack[::-1]

In [30]:
# 对比图的深度优先搜索
def graphDfsUnit(graph, u, visited, stack):
    visited[u] = True
    stack.append(u)
    for v in graph.adjtab[u]:
        if not visited[v]:
            graphDfsUnit(graph, v, visited, stack)
            
def graphDfs(graph):
    n = len(graph.adjtab)
    visited = [False for _ in range(n)]
    stack = list()
    for u in range(n):
        if not visited[u]:
            graphDfsUnit(graph, u, visited, stack)
    return stack

In [29]:
graph1 = DirectedGraph(5)
graph1.add_edge(0,1)
graph1.add_edge(0,3)
graph1.add_edge(1,3)
graph1.add_edge(1,2)
graph1.add_edge(3,2)
graph1.add_edge(3,4)
graph1.add_edge(2,4)
print('Kanhs topologic sorting:', topologicalSortingKahns(graph1))
print('DFS topologic sorting:', topologicalSortingDfs(graph1))
print('DFS traverse:', graphDfs(graph1))

Kanhs topologic sorting: [0, 1, 3, 2, 4]
DFS topologic sorting: [0, 1, 3, 2, 4]
DFS traverse: [0, 1, 3, 2, 4]


In [31]:
graph2 = DirectedGraph(6)
graph2.add_edge(4,0)
graph2.add_edge(5,0)
graph2.add_edge(5,2)
graph2.add_edge(3,1)
graph2.add_edge(2,3)
graph2.add_edge(4,1)
print('Kanhs topologic sorting:', topologicalSortingKahns(graph2))
print('DFS topologic sorting:', topologicalSortingDfs(graph2))
print('DFS traverse:', graphDfs(graph2))

Kanhs topologic sorting: [4, 5, 0, 2, 3, 1]
DFS topologic sorting: [5, 4, 2, 3, 1, 0]
DFS traverse: [0, 1, 2, 3, 4, 5]


In [9]:
# 有环图的处理。
graph3 = DirectedGraph(3)
graph3.add_edge(0,1)
graph3.add_edge(1,2)
graph3.add_edge(2,1)
# Kanhs算法可以处理有环图。
print('Kanhs topologic sorting:', topologicalSortingKahns(graph3))
# DFS对于有环图排出来的结果是错误的。
print('DFS topologic sorting:', topologicalSortingDfs(graph3))

Kanhs topologic sorting: [0]
DFS topologic sorting: [0, 1, 2]
